# Therapy Recommadation Systems for Patients 
- University of Trento
- Data Mining
- Ali Akay
- MSc. Artificial Intelligence Systems



In [1]:
import io, json
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from math import sqrt
from sklearn.metrics import mean_squared_error

In [3]:
f = open('Dataset/datasetB.json', 'r')
dataset = json.loads(f.read())
dataset

{'Conditions': [{'id': 'Cond1',
   'name': 'Abdominal aortic aneurysm',
   'type': 'Blood Diseases'},
  {'id': 'Cond2', 'name': 'Acne', 'type': 'Newborn Screening'},
  {'id': 'Cond3',
   'name': 'Acute cholecystitis',
   'type': 'Congenital and Genetic Diseases'},
  {'id': 'Cond4',
   'name': 'Acute lymphoblastic leukaemia',
   'type': 'Lung Diseases'},
  {'id': 'Cond5',
   'name': 'Acute lymphoblastic leukaemia: Children',
   'type': 'Female Reproductive Diseases'},
  {'id': 'Cond6',
   'name': 'Acute lymphoblastic leukaemia: Teenagers and young adults',
   'type': 'Nutritional diseases'},
  {'id': 'Cond7',
   'name': 'Acute myeloid leukaemia',
   'type': 'Metabolic disorders'},
  {'id': 'Cond8',
   'name': 'Acute myeloid leukaemia: Children',
   'type': 'Blood Diseases'},
  {'id': 'Cond9',
   'name': 'Acute myeloid leukaemia: Teenagers and young adults',
   'type': 'Environmental Diseases'},
  {'id': 'Cond10', 'name': 'Acute pancreatitis', 'type': 'Rare Cancers'},
  {'id': 'Cond11',


In [5]:
therapies=pd.DataFrame.from_dict(pd.json_normalize(dataset["Therapies"]), orient='columns')
therapies.columns=["therapy","name","type"]

In [6]:
df=pd.read_csv("Dataset/datamining_data.csv")
df["id"]=df.id.astype(int)

In [9]:
df.head()

,Unnamed: 0,condition,end,id_x,start,successful,therapy,cured,diagnosed,isCured,isTreated,kind,id
0,0,pc3,20120109,tr1,20111219,86.0,Th49,20120404,20111218,True,True,Cond240,0
1,1,pc3,20120217,tr2,20120203,10.0,Th45,20120404,20111218,True,True,Cond240,0
2,2,pc3,20120404,tr3,20120330,100.0,Th45,20120404,20111218,True,True,Cond240,0
3,3,pc4,19650727,tr4,19650714,100.0,Th17,19650727,19650601,True,True,Cond39,0
4,4,pc5,19731019,tr5,19730919,100.0,Th47,19731019,19730915,True,True,Cond309,0


In [10]:
#item-based 

#### Id,kind,therapy and succesful columns are used for Item-Based the Collaborative Filtering method

In [11]:
data_p=df[["id","kind","therapy","successful"]]

In [12]:
data_p["id"] = data_p["id"].astype(str) + "-" + data_p["kind"]

<ipython-input-12-e0a7b7f9e045>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_p["id"] = data_p["id"].astype(str) + "-" + data_p["kind"]


In [13]:
data_p=data_p[["id","therapy","successful"]]

#### This is the final dataset that I used. Id represent patientID-CondID. After I create matrix, I will recommend new therapies based on patientID and condID

In [14]:
data_p.head()

,id,therapy,successful
0,0-Cond240,Th49,86.0
1,0-Cond240,Th45,10.0
2,0-Cond240,Th45,100.0
3,0-Cond39,Th17,100.0
4,0-Cond309,Th47,100.0


#### I used normalized values of the succesful rate but I also tried with normal values, the rmse is the same.

In [29]:
def normalize(data):
    # compute mean rating for each user
    mean = data.groupby(by='id', as_index=False)['successful'].mean()
    norm_ratings = pd.merge(data, mean, suffixes=('','_mean'), on='id')
    
    # normalize each rating by substracting the mean rating of the corresponding user
    norm_ratings['norm_successful'] = norm_ratings['successful'] - norm_ratings['successful_mean']
    return mean.to_numpy()[:, 1], norm_ratings

In [49]:
mean, norm_ratings = normalize() 
# label_encoder object knows how to understand word labels.
label_encoder_therapy = preprocessing.LabelEncoder()

#get user mean index
mean_id=data_p.groupby(by='id', as_index=False)['successful'].mean()

# Encode labels in column 'species'.
norm_ratings['therapy']= label_encoder_therapy.fit_transform(norm_ratings['therapy'])

#norm_ratings=norm_ratings.sort_values("therapy",ascending=True)
norm_ratings=norm_ratings[["id","therapy","successful","norm_successful"]]
np_ratings = norm_ratings.to_numpy()
np_ratings

array([['0-Cond240', 43, 86.0, 20.66666666666667],
       ['0-Cond240', 39, 10.0, -55.33333333333333],
       ['0-Cond240', 39, 100.0, 34.66666666666667],
       ...,
       ['99999-Cond135', 10, 53.0, -31.33333333333333],
       ['99999-Cond135', 49, 100.0, 15.666666666666671],
       ['99999-Cond135', 49, 100.0, 15.666666666666671]], dtype=object)

In [12]:
norm_ratings

,id,therapy,successful,norm_successful
0,0-Cond240,43,86.0,20.666667
1,0-Cond240,39,10.0,-55.333333
2,0-Cond240,39,100.0,34.666667
3,0-Cond39,8,100.0,0.000000
4,0-Cond309,41,100.0,0.000000
...,...,...,...,...
1027025,99999-Cond128,45,100.0,37.333333
1027026,99999-Cond280,20,20.0,0.000000
1027027,99999-Cond135,10,53.0,-31.333333
1027028,99999-Cond135,49,100.0,15.666667


#### I created a item-user matrix

In [18]:
def rating_matrix(dataframe, column):
    crosstab = pd.crosstab(dataframe.therapy, dataframe.id, dataframe[f'{column}'], aggfunc=sum).fillna(0).values
    matrix = csr_matrix(crosstab)
    return matrix

In [14]:
R = rating_matrix(norm_ratings,"norm_successful")

#### The shape of the matrix is 51x428290 
- 51 is the number of therapy

In [15]:
R

<51x428290 sparse matrix of type '<class 'numpy.float64'>'
	with 718261 stored elements in Compressed Sparse Row format>

In [19]:
def create_model(rating_matrix, metric):
    """
    - create the nearest neighbors model with the corresponding similarity metric
    - fit the model
    """
    model = NearestNeighbors(metric=metric, n_neighbors=21, algorithm='brute')
    model.fit(rating_matrix)    
    return model
def nearest_neighbors(rating_matrix, model):
    """    
    :param rating_matrix : rating matrix of shape (nb_patient, nb_therapy)
    :param model : nearest neighbors model    
    :return
        - similarities : distances of the neighbors from the referenced patient
        - neighbors : neighbors of the referenced patient in decreasing order of similarities
    """    
    similarities, neighbors = model.kneighbors(rating_matrix)        
    return similarities[:, 1:], neighbors[:, 1:]

In [17]:
model = create_model(R, metric="cosine")
similarities, neighbors = nearest_neighbors(R, model)

In [18]:
similarities.shape

(51, 20)

In [19]:
neighbors.shape

(51, 20)

In [20]:
np_ratings

array([['0-Cond240', 43, 86.0, 20.66666666666667],
       ['0-Cond240', 39, 10.0, -55.33333333333333],
       ['0-Cond240', 39, 100.0, 34.66666666666667],
       ...,
       ['99999-Cond135', 10, 53.0, -31.33333333333333],
       ['99999-Cond135', 49, 100.0, 15.666666666666671],
       ['99999-Cond135', 49, 100.0, 15.666666666666671]], dtype=object)

In [21]:
similarities

array([[1.01571502, 1.01652327, 1.01754691, ..., 1.01927691, 1.01950254,
        1.01952189],
       [1.01555523, 1.01572963, 1.01617772, ..., 1.01924582, 1.01924653,
        1.01936117],
       [1.01551277, 1.01613496, 1.01688833, ..., 1.01907864, 1.01908559,
        1.01931387],
       ...,
       [1.01478213, 1.01602762, 1.01660466, ..., 1.01923587, 1.01939664,
        1.01968146],
       [1.01644173, 1.01656456, 1.01660466, ..., 1.0192936 , 1.01946946,
        1.01960036],
       [1.01654793, 1.01656456, 1.01665089, ..., 1.01934832, 1.01935396,
        1.01965638]])

In [20]:
def candidate_items(userid):
    """
    :param userid : user id for which we wish to find candidate items    
    :return : I_u, candidates
    """
    
    # 1. Finding the set I_u of therapy already rated by patient userid
    I_u = np_ratings[np_ratings[:, 0] == userid]
    I_u = I_u[:, 1]
    
    # 2. Taking the union of similar therapy for all therapies in I_u to form the set of candidate therapies
    c = set()
        
    for iid in I_u:    
        # add the neighbors of therapy iid in the set of candidate therapies
        c.update(neighbors[int(iid)])
        
    c = list(c)
    # 3. exclude from the set C all therapies in I_u.
    candidates = np.setdiff1d(c, I_u, assume_unique=True)
    
    return I_u, candidates

In [58]:
test_user='6-Cond248'
i_u, u_candidates = candidate_items(test_user)
print('number of therapy for the patient  : ', len(i_u))
print('number of candidate therapy for the patient : ', len(u_candidates))

number of therapy for the patient  :  1
number of candidate therapy for the patient :  20


In [21]:
def similarity_with_Iu(c, I_u):
    """
    compute similarity between an therapy c and a set of therapy I_u. For each therapy i in I_u, get similarity between 
    i and c, if c exists in the set of therapy similar to itemid.    
    :param c : itemid of a candidate therapy
    :param I_u : set of therapies already purchased by a given user    
    :return w : similarity between c and I_u
    """
    w = 0    
    for iid in I_u :        
        # get similarity between itemid and c, if c is one of the k nearest neighbors of itemid
        if c in neighbors[int(iid)] :
            w = w + similarities[int(iid), neighbors[int(iid)] == c][0]    
    return w

In [22]:
def rank_candidates(candidates, I_u):
    """
    rank candidate therapies according to their similarities with i_u    
    :param candidates : list of candidate items
    :param I_u : list of items purchased by the user    
    :return ranked_candidates : dataframe of candidate therapies, ranked in descending order of similarities with I_u
    """
    
    # list of candidate items mapped to their corresponding similarities to I_u
    sims = [similarity_with_Iu(c, I_u) for c in candidates]
    #candidates = label_encoder.inverse_transform(candidates)    
    mapping = list(zip(candidates, sims))
    
    ranked_candidates = sorted(mapping, key=lambda couple:couple[1], reverse=True)    
    return ranked_candidates

In [23]:
def topn_recommendation(userid, N=10):
    """
    Produce top-N recommendation for a given patient    
    :param userid : user for which we produce top-N recommendation
    :param n : length of the top-N recommendation list    
    :return topn
    """
    # find candidate therapy
    I_u, candidates = candidate_items(userid)
    
    # rank candidate items according to their similarities with I_u
    ranked_candidates = rank_candidates(candidates, I_u)
    
    # get the first N row of ranked_candidates to build the top N recommendation list
    topn = pd.DataFrame(ranked_candidates[:N], columns=['therapy','similarity_with_Iu'])    
    #topn = pd.merge(topn, data_p, on='CT', how='inner')    
    return topn

In [26]:
#This dataframe represents the top N recommendation list a user. These therapies are sorted in decreasing order of similarities with .
topn_recommendation(test_user)

,therapy,similarity_with_Iu
0,33,1.019669
1,16,1.019329
2,22,1.019266
3,41,1.019070
4,43,1.018984
5,10,1.018954
6,37,1.018843
7,5,1.018838
8,31,1.018631
9,39,1.018460


In [24]:
def predict(userid, itemid,mean_id):
    """
    Make rating prediction for user userid on therapy itemid    
    :param userid : id of the active patient
    :param itemid : id of the therapy for which we are making prediction        
    :return r_hat : predicted rating
    """
    
    # Get items similar to therapy itemid with their corresponding similarities
    item_neighbors = neighbors[int(itemid)]
    item_similarities = similarities[int(itemid)]

    # get ratings of user with id userid
    uratings = np_ratings[np_ratings[:, 0] == userid]
    
    # similar therapies rated by therapy the user of i
    siru = uratings[np.isin(uratings[:, 1], item_neighbors)]
    scores = siru[:, 2]
    indexes = [np.where(item_neighbors == iid)[0][0] for iid in siru[:,1]]    
    sims = item_similarities[indexes]
    
    dot = np.dot(scores, sims)
    som = np.sum(np.abs(sims))
    
    user_index=mean_id[mean_id["id"]==userid].index.values[0]

    if dot == 0 or som == 0:
        return mean[user_index]
    
    return dot / som

In [25]:
def topn_prediction(userid,mean_id):
    """
    :param userid : id of the active patient    
    :return topn : initial topN recommendations returned by the function item2item_topN
    :return topn_predict : topN recommendations reordered according to rating predictions
    """
    # make top N recommendation for the active user
    topn = topn_recommendation(userid)
    
    # get list of therapies of the top N list
    itemids = topn.therapy.to_list()
    
    predictions = []
    
    # make prediction for each therapies in the top N list
    for itemid in itemids:
        r = predict(userid, itemid,mean_id)
        
        predictions.append((itemid,r))
    
    predictions = pd.DataFrame(predictions, columns=['therapy','prediction'])
    
    # merge the predictions to topN_list and rearrange the list according to predictions
    topn_predict = pd.merge(topn, predictions, on='therapy', how='inner')
    topn_predict = topn_predict.sort_values(by=['similarity_with_Iu'], ascending=False)
    topn_predict["therapy"]=label_encoder_therapy.inverse_transform(topn_predict["therapy"])    
    
    
    return topn, topn_predict

In [51]:
topn, topn_predict = topn_prediction(userid='6-Cond248',mean_id=mean_id)
topn_predict

,therapy,similarity_with_Iu,prediction
0,Th4,1.019669,23.0
1,Th24,1.019329,23.0
2,Th3,1.019266,23.0
3,Th47,1.019070,23.0
4,Th49,1.018984,23.0
5,Th19,1.018954,23.0
6,Th43,1.018843,23.0
7,Th14,1.018838,23.0
8,Th38,1.018631,23.0
9,Th45,1.018460,23.0


### Test 

#### For the last part of the code, I predicted a recommadation for the test dataset, but for the condition "pc277636" does not exist in my dataset so I couldn't find a recommadation for this patient and condition.

In [30]:
test_data=pd.read_csv('datasetB_cases.txt', delimiter = "\t")
test_data=test_data.reset_index()
test_data["PatientID"]=test_data["index"]
test_data.drop("index",inplace=True,axis=1)
test_data.columns=["id","condition"]
test_data=test_data.merge(df[["id","condition","kind"]],on=["id","condition"],how="left").drop_duplicates()
test_data["id"]=test_data["id"].astype(str)+"-"+test_data["kind"]
test_data.fillna("9999999-Cond99999",inplace=True)
test_list=test_data["id"].to_list()

In [31]:
test_data.head()

,id,condition,kind
0,6-Cond248,pc32,Cond248
1,9999999-Cond99999,pc277636,9999999-Cond99999
2,82486-Cond16,pc445475,Cond16
9,51348-Cond216,pc277652,Cond216
14,51358-Cond120,pc277696,Cond120


In [32]:
def test(test_list):

  patient=[]
  condition=[]
  prediction_therapy=[]
  prediction_name=[]

  for i in test_list:
      topn, topn_predict = topn_prediction(userid=i)
      pred=pd.merge(topn_predict,therapies,on="therapy",how="left")[["therapy","name"]]
      patient.append(i.split("-")[0])
      condition.append(i.split("-")[1])
      prediction_therapy.append(pred["therapy"].to_list())
      prediction_name.append(pred["name"].to_list())
  pred=pd.DataFrame([patient,condition,prediction_therapy,prediction_name]).T
  pred.columns=["PatientId","condition","Recommended_Therapy","Recommended_Therapy_name"]
  results=pred.to_dict('records')

  with io.open('test_results.txt', 'w', encoding='utf-8') as f:
    f.write(json.dumps(results, ensure_ascii=False))

  return results


In [33]:
test(test_list)    

[{'PatientId': '6',
  'condition': 'Cond248',
  'Recommended_Therapy': ['Th4',
   'Th24',
   'Th3',
   'Th47',
   'Th49',
   'Th19',
   'Th43',
   'Th14',
   'Th38',
   'Th45'],
  'Recommended_Therapy_name': ['aurotherapy',
   'intravenous immunoglobulin',
   'aquarium therapy',
   'stepdown therapy',
   'systemic therapy',
   'herbal therapy',
   'respiratory therapy',
   'electromagnetic therapy (alternative medicine)',
   'polytherapy',
   'speech therapy']},
 {'PatientId': '9999999',
  'condition': 'Cond99999',
  'Recommended_Therapy': [],
  'Recommended_Therapy_name': []},
 {'PatientId': '82486',
  'condition': 'Cond16',
  'Recommended_Therapy': ['Th17',
   'Th13',
   'Th12',
   'Th10',
   'Th8',
   'Th18',
   'Th7',
   'Th29',
   'Th25',
   'Th40'],
  'Recommended_Therapy_name': ['exercise therapy',
   'drug therapy',
   'dietary therapy',
   'curative therapy',
   'counseling',
   'gold standard therapy',
   'consolidation therapy',
   'magnetic resonance therapy',
   'investiga

### Evaluate the RMSE

In [26]:
def get_examples(dataframe, labels_column="successful"):
    examples = dataframe[['id', 'therapy']].values
    labels = dataframe[f'{labels_column}'].values
    return examples, labels

In [58]:
def train_test_split(examples, labels, test_size=0.1, verbose=0):
    if verbose:
        print("Train/Test split ")
        print(100-test_size*100, "% of training data")
        print(test_size*100, "% of testing data")    

    # split data into train and test sets
    train_examples, test_examples, train_labels, test_labels = sklearn_train_test_split(
        examples, 
        labels, 
        test_size=0.04, 
        random_state=42, 
        shuffle=True
    )

    # transform train and test examples to their corresponding one-hot representations
    train_users = train_examples[:, 0]
    test_users = test_examples[:, 0]

    train_items = train_examples[:, 1]
    test_items = test_examples[:, 1]

    # Final training and test set
    x_train = np.array(list(zip(train_users, train_items)))
    x_test = np.array(list(zip(test_users, test_items)))

    y_train = train_labels
    y_test = test_labels

    if verbose:
        print()
        print('number of training examples : ', x_train.shape)
        print('number of training labels : ', y_train.shape)
        print('number of test examples : ', x_test.shape)
        print('number of test labels : ', y_test.shape)

    return (x_train, x_test), (y_train, y_test)

In [59]:
# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(norm_ratings, labels_column='successful')
# train test split
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

In [60]:
def evaluate(x_test, y_test):
        print('Evaluate the model on {} test data ...'.format(x_test.shape[0]))
        preds = list(predict(u, int(i),mean_id) for (u, i) in x_test)
        rmse = sqrt(mean_squared_error(y_test,np.array(preds)))
        print('\nRMAE :', rmse)
        return rmse

In [56]:
evaluate(x_test, y_test)

Evaluate the model on 41082 test data ...

RMAE : 36.383018520024336


36.383018520024336

## Experiment 2 patients  selected who had at least one trial for same condition and whose most recent trial had ended with a higher succesful rate

In [61]:
data_p.head()

,id,therapy,successful
0,0-Cond240,Th49,86.0
1,0-Cond240,Th45,10.0
2,0-Cond240,Th45,100.0
3,0-Cond39,Th17,100.0
4,0-Cond309,Th47,100.0


In [62]:
data_exp=data_p["id"]+","+data_p["therapy"]
data_exp=pd.DataFrame(data_exp)
data_exp["successful"]=data_p["successful"]
data_exp.columns=["id","successful"]
data_exp_1=pd.DataFrame(data_exp.groupby("id").successful.max()).reset_index()
dt=data_exp_1["id"].str.split(",", n = 1, expand = True)
dt["successful"]=data_exp_1["successful"]
dt.columns=["id","therapy","successful"]

In [63]:
dt

,id,therapy,successful
0,0-Cond240,Th45,100.0
1,0-Cond240,Th49,86.0
2,0-Cond280,Th19,59.0
3,0-Cond280,Th29,56.0
4,0-Cond280,Th3,30.0
...,...,...,...
918298,99999-Cond280,Th28,20.0
918299,99999-Cond319,Th6,100.0
918300,99999-Cond5,Th6,100.0
918301,99999-Cond51,Th11,100.0


In [64]:
mean, norm_ratings = normalize(dt) 
# label_encoder object knows how to understand word labels.
label_encoder_therapy = preprocessing.LabelEncoder()

#get user mean index
mean_id=dt.groupby(by='id', as_index=False)['successful'].mean()

# Encode labels in column 'species'.
norm_ratings['therapy']= label_encoder_therapy.fit_transform(norm_ratings['therapy'])

#norm_ratings=norm_ratings.sort_values("therapy",ascending=True)
norm_ratings=norm_ratings[["id","therapy","successful","norm_successful"]]
np_ratings = norm_ratings.to_numpy()
np_ratings

array([['0-Cond240', 39, 100.0, 7.0],
       ['0-Cond240', 43, 86.0, -7.0],
       ['0-Cond280', 10, 59.0, 6.0],
       ...,
       ['99999-Cond5', 47, 100.0, 0.0],
       ['99999-Cond51', 2, 100.0, 0.0],
       ['99999-Cond7', 4, 100.0, 0.0]], dtype=object)

In [65]:
R = rating_matrix(norm_ratings,"successful")

In [66]:
model = create_model(R, metric="cosine")
similarities, neighbors = nearest_neighbors(R, model)

In [67]:
# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(norm_ratings, labels_column='norm_successful')
# train test split
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

In [68]:
evaluate(x_test, y_test)

Evaluate the model on 36733 test data ...

RMAE : 80.35910075641252


80.35910075641252